In [ ]:
!pip install pyspark==3.3.1 py4j==0.10.9.5

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [1]:
from pyspark.sql import SparkSession
from pyspark import SparkConf

conf = SparkConf()
conf.set("spark.app.name", "PySpark DataFrame #3")
conf.set("spark.master", "local[*]")

spark = SparkSession.builder\
        .config(conf=conf)\
        .getOrCreate()

In [2]:
!wget https://s3-geospatial.s3.us-west-2.amazonaws.com/transfer_cost.txt

'wget'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [3]:
!ls -tl

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [4]:
!head -5 transfer_cost.txt

'head'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [5]:
import pyspark.sql.functions as F
from pyspark.sql.types import *

schema = StructType([ StructField("text", StringType(), True)])

# 텍스트를 로드
transfer_cost_df = spark.read.schema(schema).text("./data/transfer_cost.txt")

In [6]:
# truncate = False면 기본적으로 필드의 크기가 어느 선을 넘어가면 잘려서 표시되는데, 그냥 다 보여달라는 뜻
transfer_cost_df.show(truncate=False)

+---------------------------------------------------------------------------+
|text                                                                       |
+---------------------------------------------------------------------------+
|On 2021-01-04 the cost per ton from 85001 to 85002 is $28.32 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85004 is $25.68 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85007 is 19.86 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85007 is 20.52 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85010 is 20.72 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85012 is $18.98 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85001 to 85013 is 26.64 at Haul Today  |
|On 2021-01-04 the cost per ton from 85001 to 85020 is 26.34 at ABC Hauling |
|On 2021-01-04 the cost per ton from 85001 to 85021 is $20.15 at ABC Hauling|
|On 2021-01-04 the cost per ton from 85002 to 85001 is 21.57 at 

In [7]:
from pyspark.sql.functions import *
regex_str = r'On (\S+) the cost per ton from (\d+) to (\d+) is (\S+) at (.*)'

# 데이터프레임이 지원하는 Function 중에
# withColumn이라는 게 있다.
# 추가하려는 컬럼 혹은, 존재하는 컬럼의 이름이 되고,
# 그 다음의 내용은 그 컬럼의 value가 된다.
# user defined function 사용이 가능하다.
# sql.functions 안에 regex_extract가 있는데, 이는 정규표현식 추출 관련 함수
# 정의한 것을 지정하면 되고, 매칭 된 것 중에 몇번째로 할 건지 숫자는 1부터 센다.

df_with_new_columns = transfer_cost_df\
    .withColumn('week', regexp_extract('text', regex_str, 1))\
    .withColumn('departure_zipcode', regexp_extract(column('text'), regex_str, 2))\
    .withColumn('arrival_zipcode', regexp_extract(transfer_cost_df.text, regex_str, 3))\
    .withColumn('cost', regexp_extract(col('text'), regex_str, 4))\
    .withColumn('vendor', regexp_extract(col('text'), regex_str, 5))

In [8]:
df_with_new_columns.printSchema()

root
 |-- text: string (nullable = true)
 |-- week: string (nullable = true)
 |-- departure_zipcode: string (nullable = true)
 |-- arrival_zipcode: string (nullable = true)
 |-- cost: string (nullable = true)
 |-- vendor: string (nullable = true)



In [9]:
# text라는 컬럼 Drop
final_df = df_with_new_columns.drop("text")

In [11]:
import findspark
findspark.init()

final_df.write.csv("extracted.csv")
# final_df.toPandas().to_csv('extracted.csv')

In [12]:
!ls -tl

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [13]:
!ls -tl extracted.csv/

'ls'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [14]:
!head -5 extracted.csv/part-00000-a909db0a-d743-4a0c-96fc-60c1a8eef076-c000.csv

'head'��(��) ���� �Ǵ� �ܺ� ����, ������ �� �ִ� ���α׷�, �Ǵ�
��ġ ������ �ƴմϴ�.


In [15]:
final_df.write.format("json").save("extracted.json")

In [ ]:
!ls -tl extracted.json/

total 428
-rw-r--r-- 1 root root      0 Jan 15 22:00 _SUCCESS
-rw-r--r-- 1 root root 436305 Jan 15 22:00 part-00000-104f95b9-f2c6-4f77-a170-583c78106e11-c000.json


In [ ]:
!head -1 extracted.json/part-00000-104f95b9-f2c6-4f77-a170-583c78106e11-c000.json

{"week":"2021-01-04","departure_zipcode":"85001","arrival_zipcode":"85002","cost":"$28.32","vendor":"ABC Hauling"}
